# ASK

## Description du projet : Analyse exploratoire de données Marketing

### **1. Contexte**
Le projet se déroule dans un **secteur bancaire de plus en plus concurrentiel**. Pour garantir sa rentabilité et améliorer l'expérience de ses clients, une institution bancaire portugaise cherche à optimiser l'efficacité de ses campagnes de **marketing direct** (appels téléphoniques). L'enjeu est de passer d'une approche massive à un ciblage stratégique basé sur les données.

### **2. Problématique**
La banque doit déterminer **quels clients cibler en priorité** pour maximiser les souscriptions aux **dépôts à terme**. D'un point de vue technique, il s'agit d'un problème de **classification** : il faut prédire si la variable cible « y » (souscription du client) sera « yes » ou « no » en fonction du profil du client et des interactions passées.

### **3. Objectifs**
*   **Analyse exploratoire (EDA) :** Explorer les données pour extraire des insights stratégiques.
*   **Validation statistique :** Appliquer des méthodes avancées (ANOVA, Khi-deux, etc.) pour valider les résultats de manière objective.
*   **Modélisation prédictive :** Développer un modèle simple (type KNN) pour prédire l'acceptation des clients.
*   **Aide à la décision :** Fournir des recommandations concrètes et des scénarios pour maximiser le retour sur investissement.

### **4. Besoins**
*   **Données :** Un jeu de données de **45 211 instances** comportant **16 variables** telles que l'âge, l'emploi, le solde bancaire et l'historique des contacts.
*   **Méthodologie :** Application de la démarche **CRISP-DM** (Compréhension, Préparation, Analyse, Modélisation, Évaluation, Recommandations).
*   **Outils technologiques :**
    *   **Langage :** Python.
    *   **Bibliothèques :** Pandas, NumPy, Scikit-Learn, Matplotlib, Seaborn.
    *   **Optionnel :** Streamlit pour la création d'un tableau de bord interactif.
*   **Livrables :** Un rapport d'analyse, le code source documenté (Notebooks) et les recommandations stratégiques.

# PREPARE

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [30]:
df_bank_marketing_raw = pd.read_csv("../data/raw/bank-full.csv", encoding="ISO-8859-1",sep=";")

In [31]:
df_bank_marketing_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


### Dictionnaire de données

| Variable | Type | Description |
|---------|------|-------------|
| age | int64 | Âge du client (en années, variable numérique continue) |
| job | object | Type d’emploi du client (catégorielle) |
| marital | object | Statut matrimonial du client |
| education | object | Niveau d’éducation du client |
| default | object | Indique si le client est en défaut de crédit (yes / no) |
| balance | int64 | Solde annuel moyen du compte bancaire du client (en euros) |
| housing | object | Indique si le client possède un prêt immobilier (yes / no) |
| loan | object | Indique si le client possède un prêt personnel (yes / no) |
| contact | object | Type de communication utilisé lors du dernier contact |
| day | int64 | Jour du mois du dernier contact |
| month | object | Mois du dernier contact |
| duration | int64 | Durée du dernier contact avec le client (en secondes) |
| campaign | int64 | Nombre de contacts effectués durant la campagne actuelle |
| pdays | int64 | Nombre de jours depuis le dernier contact (-1 si jamais contacté) |
| previous | int64 | Nombre de contacts effectués avant la campagne actuelle |
| poutcome | object | Résultat de la campagne marketing précédente |
| y | object | Variable cible : souscription à un dépôt à terme (yes / no) |

In [32]:
df_bank_marketing_raw.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [33]:
df_bank_marketing_raw.shape

(45211, 17)

In [34]:
df_bank_marketing_raw.columns.tolist()

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [58]:
# Affiche le nombre de lignes dupliquées dans le DataFrame
print(f"Nombre de doublons : {int(df_bank_marketing_raw.duplicated().sum())}")

Nombre de doublons : 0


### PROCESS

In [35]:
# Un renommage court et efficace
dict_rename = {
    'job': 'metier',
    'marital': 'etat_civil',
    'education': 'niveau_education',
    'default': 'defaut_credit',
    'balance': 'solde_annuel_moyen',
    'housing': 'pret_immobilier',
    'loan': 'pret_personel',
    'contact': 'type_contact',
    'day': 'jour',
    'month': 'mois',
    'duration': 'duree_appel',
    'campaign': 'nb_appels_campagne',
    'pdays': 'jours_depuis_dernier',
    'previous': 'appels_precedents',
    'poutcome': 'resultat_prec',
    'y': 'cible'
}

# Application immédiate
df_bank_marketing_raw.rename(columns=dict_rename, inplace=True)


In [38]:
df_bank_marketing_raw.columns

Index(['age', 'metier', 'etat_civil', 'niveau_education', 'defaut_credit',
       'solde_annuel_moyen', 'pret_immobilier', 'pret_personel',
       'type_contact', 'jour', 'mois', 'duree_appel', 'nb_appels_campagne',
       'jours_depuis_dernier', 'appels_precedents', 'resultat_prec', 'cible'],
      dtype='object')

In [44]:
from src.common import *

cat_features, cont_features = identify_features(df_bank_marketing_raw)

print(f"✅ Variables Catégorielles ({len(cat_features)}) : {cat_features}\n")
print(f"📈 Variables Continues ({len(cont_features)}) : {cont_features}")

✅ Variables Catégorielles (7) : ['etat_civil', 'niveau_education', 'defaut_credit', 'pret_immobilier', 'pret_personel', 'type_contact', 'resultat_prec']

📈 Variables Continues (9) : ['age', 'metier', 'solde_annuel_moyen', 'jour', 'mois', 'duree_appel', 'nb_appels_campagne', 'jours_depuis_dernier', 'appels_precedents']


In [54]:
df_bank_marketing_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
age,45211.0,40.936210,10.618762,18.0,33.0,39.0,48.0,95.0
solde_annuel_moyen,45211.0,1362.272058,3044.765829,-8019.0,72.0,448.0,1428.0,102127.0
jour,45211.0,15.806419,8.322476,1.0,8.0,16.0,21.0,31.0
duree_appel,45211.0,258.163080,257.527812,0.0,103.0,180.0,319.0,4918.0
nb_appels_campagne,45211.0,2.763841,3.098021,1.0,1.0,2.0,3.0,63.0
jours_depuis_dernier,45211.0,40.197828,100.128746,-1.0,-1.0,-1.0,-1.0,871.0
appels_precedents,45211.0,0.580323,2.303441,0.0,0.0,0.0,0.0,275.0


In [53]:
round(float(df_bank_marketing_raw['age'].mean()))

41